<a href="https://colab.research.google.com/github/Mohamed-Khalil67/Project_SARS-COV-2/blob/main/SARS_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *